In [0]:
import io
import sys
import random
import string
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
def load_text(filename):
    with open(filename, 'r') as f:
        text = f.read()
    return text
 
file_poem = 'drive/shakespeare_poems.txt' # Path of the file
text = load_text(file_poem)
print(text[:200])


THE SONNETS

by William Shakespeare

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his mem


In [0]:
chars = sorted(list(set(text)))
print('Number of distinct characters:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Number of distinct characters: 61


In [0]:
max_len_chars = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - max_len_chars, step):
    sentences.append(text[i: i + max_len_chars])
    next_chars.append(text[i + max_len_chars])
print('nb sequences:', len(sentences))

nb sequences: 31327


In [0]:
x = np.zeros((len(sentences), max_len_chars, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
model = Sequential()
model.add(GRU(128, input_shape=(max_len_chars, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.fit(x, y,batch_size=128,epochs=10)
model.save("poem_gen_model.h5")

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
31327/31327 [==============================] - 29s 912us/step - loss: 2.2775
Epoch 2/10
31327/31327 [==============================] - 28s 894us/step - loss: 1.9028
Epoch 3/10
31327/31327 [==============================] - 28s 896us/step - loss: 1.7642
Epoch 4/10
31327/31327 [==============================] - 28s 894us/step - loss: 1.6717
Epoch 5/10
31327/31327 [==============================] - 28s 895us/step - loss: 1.6048
Epoch 6/10
31327/31327 [==============================] - 29s 910us/step - loss: 1.5464
Epoch 7/10
31327/31327 [==============================] - 28s 904us/step - loss: 1.5048
Epoch 8/10
31327/31327 [==============================] - 28s 906us/step - loss: 1.4712
Epoch 9/10
31327/31327 [==============================] - 28s 895us/step - loss: 1.4451
Epoch 10/10
31327/31327 [==============================] - 28s 894us/step - loss: 1.4324


In [0]:
from keras.models import load_model
model_loaded = load_model('poem_gen_model.h5')

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def generate_poem(model, num_chars_to_generate=400):
    start_index = random.randint(0, len(text) - max_len_chars - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len_chars]
    generated += sentence
    print("Seed sentence: {}".format(generated))
    for i in range(num_chars_to_generate):
        x_pred = np.zeros((1, max_len_chars, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 1)
        next_char = indices_char[next_index]
 
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [0]:
generate_poem(model_loaded, 100)

Seed sentence: my sake,
So him I lose through my unkind


'my sake,\nSo him I lose through my unkind wide upen\nis imine.\nWorlets from nor thy sfullight arpain flade,\nIn sthat fitt, row and hears lors '